In [1]:
# -*- coding: utf-8 -*-
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import matplotlib.pyplot as plt
import plotly.graph_objs as go
plt.style.use('ggplot')
import seaborn as sns
import pandas as pd
import plotly as py
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# import datas
aerial = pd.read_csv('operations.csv')
weather = pd.read_csv('Summary of Weather.csv')
weather_station_location = pd.read_csv('Weather Station Locations.csv')

In [2]:
# drop countries that are NaN
aerial = aerial[pd.isna(aerial.Country)==False]
# drop if target longitude is NaN
aerial = aerial[pd.isna(aerial['Target Longitude'])==False]
# Drop if takeoff longitude is NaN
aerial = aerial[pd.isna(aerial['Takeoff Longitude'])==False]
# drop unused features
drop_list = ['Mission ID','Unit ID','Target ID','Altitude (Hundreds of Feet)','Airborne Aircraft',
             'Attacking Aircraft', 'Bombing Aircraft', 'Aircraft Returned',
             'Aircraft Failed', 'Aircraft Damaged', 'Aircraft Lost',
             'High Explosives', 'High Explosives Type','Mission Type',
             'High Explosives Weight (Pounds)', 'High Explosives Weight (Tons)',
             'Incendiary Devices', 'Incendiary Devices Type',
             'Incendiary Devices Weight (Pounds)',
             'Incendiary Devices Weight (Tons)', 'Fragmentation Devices',
             'Fragmentation Devices Type', 'Fragmentation Devices Weight (Pounds)',
             'Fragmentation Devices Weight (Tons)', 'Total Weight (Pounds)',
             'Total Weight (Tons)', 'Time Over Target', 'Bomb Damage Assessment','Source ID']
aerial.drop(drop_list, axis=1,inplace = True)
aerial = aerial[aerial.iloc[:,8]!="4248"] # drop this takeoff latitude 
aerial = aerial[aerial.iloc[:,9]!="1355"]   # drop this takeoff longitude
weather_station_location = weather_station_location.loc[:,["WBAN","NAME","STATE/COUNTRY ID","Latitude","Longitude"] ]
weather = weather.loc[:,["STA","Date","MeanTemp"] ]

In [3]:
# Takeoff and Base informations
aerial["color"] = ""
aerial.color[aerial.Country == "USA"] = "rgb(0,116,217)"
aerial.color[aerial.Country == "GREAT BRITAIN"] = "rgb(255,65,54)"
aerial.color[aerial.Country == "NEW ZEALAND"] = "rgb(133,20,75)"
aerial.color[aerial.Country == "SOUTH AFRICA"] = "rgb(255,133,27)"

data = [
    dict(
        type='scattergeo',
        lon = aerial['Takeoff Longitude'],
        lat = aerial['Takeoff Latitude'],
        text = "Country: " + aerial.Country + " Takeoff Location: "+aerial["Takeoff Location"]+" Takeoff Base: " + aerial['Takeoff Base'],
        hoverinfo = 'text',
        mode = 'markers',
        marker = dict(
            sizemode = 'area',
            sizeref = 1,
            size= 7 ,
            line = dict(width=1,color = "white"),
            color = aerial["color"],
            opacity = 0.7)
    )
]
layout = dict(
    title = dict(
        text = '<b> Countries Take Off Bases </b>',
        x = 0.5, y = 0.9,
        font = dict(
            family = 'Italic',
            size = 20,
            color = 'Black')
    ),
    hovermode='closest',
    geo = dict(
        showframe=False, showland=True, showcoastlines=True, showcountries=True,
        countrywidth=2, projection=dict(type='mercator'),
        landcolor = 'rgb(217, 217, 217)',
        subunitwidth=1,
        showlakes = True,
        lakecolor = 'rgb(255, 255, 255)',
        countrycolor='rgb(5, 5, 5)'
    )
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [7]:
# Attack Target
airports = [
    dict(
        type='scattergeo',
        lon = aerial['Takeoff Longitude'],
        lat = aerial['Takeoff Latitude'],
        text = "Country: " + aerial.Country + " Takeoff Location: "+aerial["Takeoff Location"]+" Takeoff Base: " + aerial['Takeoff Base'],
        hoverinfo = 'text',
        mode = 'markers',
        marker = dict(
            size= 7 ,
            color = aerial["color"],
            line = dict(width=1,color = "white"),
            opacity = 0.7)
    )
]

targets = [
    dict(
        type='scattergeo',
        lon = aerial['Takeoff Longitude'],
        lat = aerial['Takeoff Latitude'],
        text = "Target Country: " + aerial['Target Country']+ " Target City: " + aerial["Target City"],
        hoverinfo = 'text',
        mode = 'markers',
        marker = dict(
            size= 1 ,
            color = 'red',
            line = dict(width=0.5,color = "red"),
            opacity = 0.7)
    )
]

flight_paths = []
for i in range(len(aerial['Target Longitude'])):
    flight_paths.append(
        dict(
            type='scattergeo',
            lon = [aerial.iloc[i,9], aerial.iloc[i,16]],
            lat = [aerial.iloc[i,8], aerial.iloc[i,15]],
            mode = 'lines',
            line = dict(width=0.7,color = "black"),
            opacity = 0.6
        )
    )
layout = dict(
    title = dict(
        text = "<b> Attacker Country's Target </b>",
        x = 0.5, y = 0.9,
        font = dict(
            family = 'Italic',
            size = 20,
            color = 'Black')
    ),
    hovermode='closest',
    geo = dict(
        showframe=False, showland=True, showcoastlines=True, showcountries=True,
        countrywidth=1, projection=dict(type='mercator'),
        landcolor = 'rgb(217, 217, 217)',
        subunitwidth=1,
        showlakes = True,
        lakecolor = 'rgb(255, 255, 255)',
        countrycolor='rgb(5, 5, 5)'
    )
)
fig = dict(data=flight_paths + airports+targets, layout=layout)
iplot(fig)